In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!python -m pip install detectron2 -f \
  https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.6/index.html

# Importing Libraries

In [ ]:
from bs4 import BeautifulSoup as bs
#cuda 10.1
import torch, torchvision
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import glob

import os
import ntpath
import numpy as np
import cv2
import random
import itertools
import pandas as pd
from tqdm import tqdm
import urllib
import json
import PIL.Image as Image

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.structures import BoxMode

In [ ]:
import shutil

path_cpy = "/kaggle/input/face-mask-detection"

# shutil.copytree(path_cpy)

# Helper functions

In [ ]:
def gen_box(obj):
#     Getting bounding box coordinates
    xmin = int(obj.find('xmin').text)
    xmax = int(obj.find('xmax').text)
    ymin = int(obj.find('ymin').text)
    ymax = int(obj.find('ymax').text)
    
    return [xmin, ymin, xmax, ymax]

def gen_label(obj):
    if obj.find('name').text == "with_mask":
        return "mask" #     for without_mask label set to 1
    elif obj.find('name').text == "mask_weared_incorrect":
        return "incorrect_mask"     # for mask_weared_incorrect label set to 2
    return "no_mask" #     for without_mask label set to 0

# Getting Data from xml file

In [ ]:
coordinates = []
labels = []
name= []
size = []
def gen_list(path):
    xml_list  = list(sorted(os.listdir(path+"/annotations")))
    img_list = list(sorted(os.listdir(path+"/images")))
    total = len(img_list)
    assert len(img_list) == len(xml_list)
    counter = 0
    for xml in xml_list:
        with open(path+f"/annotations/{xml}") as file:
            data = file.read()
            soup = bs(data, "xml")
            xml_objs = soup.find_all('object')
            counter +=1
            for i in xml_objs:
                coordinates.append(gen_box(i)) 
                labels.append(gen_label(i))
                name.append(soup.find('filename').text)
        print(f"file processed {counter} / {total}...")

In [ ]:
gen_list(path_cpy)

In [ ]:
xml_list  = list(sorted(os.listdir(path_cpy+"/annotations")))

# Making DataFrame from data extracted from XML

In [ ]:
df = pd.DataFrame()

In [ ]:
df['name'] = name
df['labels'] = labels
# initializing labels
df['xmin'] = 0
df['ymin'] = 0
df['xmax'] = 0
df['ymax'] = 0
df['height'] = np.zeros(len(df['name']))
df['width'] = np.zeros(len(df['name']))

# Assigning values to columns

In [ ]:
for i,(xmin,ymin,xmax,ymax) in enumerate(coordinates):
    df['xmin'][i] = xmin
    df['ymin'][i] = ymin
    df['xmax'][i] = xmax
    df['ymax'][i] = ymax


for i,fname in enumerate(name):
    img = cv2.imread(path_cpy+'/images/'+fname)
    height = img.shape[0]
    width = img.shape[1]
    df['height'][i] = height
    df['width'][i] = width

In [ ]:
df.head()

# Splitting Data into Train and Test set

In [ ]:
unique_files = df.name.unique()

train_files = set(np.random.choice(unique_files, int(len(unique_files) * 0.95), replace=False))
train_df = df[df.name.isin(train_files)]
test_df = df[~df.name.isin(train_files)]
classes = df.labels.unique().tolist()

In [ ]:
classes

# Creating function that formats data in COCO Dateset format

In [ ]:
def my_dataset_function(df,classes):
    data_list = []
    for i,fname in enumerate(df['name'].unique()):
        record = dict()
        #gtetting all rows with nmae as fnmae
        image_df = df[df['name']==fname]
        #image path
        img_path = path_cpy +'/images/'+fname
        #entring recprds
        record['file_name'] = img_path
        record['image_id'] = i
        record['height'] = int(image_df.iloc[0].height)
        record['width'] = int(image_df.iloc[0].width)
        
        objs = []
        
        for _,row in image_df.iterrows():
            
        
            xmin = int(row.xmin)
            ymin = int(row.ymin)
            xmax = int(row.xmax)
            ymax = int(row.ymax)

            poly = [
            (xmin, ymin), (xmax, ymin),
            (xmax, ymax), (xmin, ymax)
            ]
            poly = list(itertools.chain.from_iterable(poly))

            obj = {
            "bbox": [xmin, ymin, xmax, ymax],
            "bbox_mode": BoxMode.XYXY_ABS,
            "segmentation": [poly],
            "category_id": classes.index(row.labels),
            "iscrowd": 0
            }
            objs.append(obj)

        record["annotations"] = objs
        data_list.append(record)
    return data_list

          
        

In [ ]:
len(train_df),len(test_df)

# Registering Datasets

In [ ]:
for d in ["train", "val"]:
  DatasetCatalog.register("data_" + d, lambda d=d: my_dataset_function(train_df if d == "train" else test_df, classes))
  MetadataCatalog.get("data_" + d).set(thing_classes=classes)

statement_metadata = MetadataCatalog.get("data_train")

In [ ]:
statement_metadata

In [ ]:
class CocoTrainer(DefaultTrainer):

  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=None):

    if output_folder is None:
        os.makedirs("coco_eval", exist_ok=True)
        output_folder = "coco_eval"

    return COCOEvaluator(dataset_name, cfg, False, output_folder)

# Initializing and setting cfg

In [ ]:
cfg = get_cfg()

cfg.merge_from_file(
  model_zoo.get_config_file(
    "COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml"
  )
)

cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(
  "COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml"
)
cfg.DATASETS.TRAIN = ("data_train",)
cfg.DATASETS.TEST = ("data_val",)
cfg.DATALOADER.NUM_WORKERS = 4
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.WARMUP_ITERS = 1000
cfg.SOLVER.MAX_ITER = 1500
cfg.SOLVER.STEPS = (1000, 1500)
cfg.SOLVER.GAMMA = 0.05
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(classes)
cfg.TEST.EVAL_PERIOD = 500

# Training Detectron2 

In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

trainer = CocoTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

# Saving model

In [ ]:
torch.save(trainer.model, 'checkpoint.pth')

# Let's try our model!

In [ ]:
im = cv2.imread('../input/face-mask-detection/images/maksssksksss110.png')

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.85
predictor = DefaultPredictor(cfg)

In [ ]:
evaluator = COCOEvaluator("data_val", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "data_val")
inference_on_dataset(trainer.model, val_loader, evaluator)

In [ ]:
outputs = predictor(im)
v = Visualizer(
im[:, :, ::-1],
metadata=statement_metadata,
scale=1.,
instance_mode=ColorMode.IMAGE
)
instances = outputs["instances"].to("cpu")
instances.remove('pred_masks')
v = v.draw_instance_predictions(instances)
result = v.get_image()[:, :, ::-1]
file_name = ntpath.basename('test')

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(13,10))
plt.imshow(result)

In [ ]:
os.listdir('./output')

In [ ]:
shutil.copy('./output/model_final.pth','./')